In [7]:
!pip install -qqq -r /content/requirements.txt

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement python-magic-bin==0.4.14 (from versions: none)
ERROR: No matching distribution found for python-magic-bin==0.4.14


In [11]:
!pip install langchain_groq langchain streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.9 MB/s eta 0:00:00
  Using cached GitPython-3.1.42-py3-none-any.whl (195 kB)
  Using cached pydeck-0.8.1b0-py2.py3-none-any.whl (4.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.7 MB/s eta 0:00:00
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)


In [18]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.0 MB/s eta 0:00:00


In [2]:
PROMPT_TEMPLATE = """
You are a great researcher. With the information provided understand in deep and try to answer the question.
If you cant answer the question based on the information either say you cant find an answer or unable to find an answer.
So try to understand in depth about the context and answer only based on the information provided. Dont generate irrelevant answers.

Context: {context}
Question: {question}
Do provide only helpful answers

Answer:
"""
INPUT_VARIABLES = ["context", "question"]
SEPARATORS = "\n"
CHUNK_SIZE = 10000
CHUNK_OVERLAP = 1000
EMBEDDER = "BAAI/bge-base-en-v1.5"
CHAIN_TYPE = "stuff"
SEARCH_KWARGS = {'k': 3}

In [9]:
import os
#from dotenv import load_dotenv, find_dotenv
import json
import requests
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders.url import UnstructuredURLLoader
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
#load_dotenv(find_dotenv())


import os

# Set environment variables
os.environ['SERPER_API_KEY'] = '68aa9b702d21872906990b6161f93a57729d4ec1'
os.environ['GROQ_API_KEY'] = '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'


class Researcher:

    def __init__(self):
        self.serper_api_key = os.getenv("SERPER_API_KEY")
        self.groq_api_key = os.getenv("GROQ_API_KEY")
        self.prompt_template = PromptTemplate(
            template=PROMPT_TEMPLATE,
            input_variables=INPUT_VARIABLES
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=SEPARATORS,
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )
        self.llm = ChatGroq(temperature=0.5, model_name="mixtral-8x7b-32768", groq_api_key=self.groq_api_key)
        self.hfembeddings = HuggingFaceEmbeddings(
                            model_name=EMBEDDER,
                            model_kwargs={'device': 'cuda'}
                        )

    def search_articles(self, query):

        url = "https://google.serper.dev/search"
        data = json.dumps({"q":query})

        headers = {
            'X-API-KEY': self.serper_api_key,
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=data)

        return response.json()

    def research_answerer(self):

        research_qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type=CHAIN_TYPE,
                retriever= self.db.as_retriever(search_kwargs=SEARCH_KWARGS),
                return_source_documents=True,
                verbose=True,
                chain_type_kwargs={"prompt": self.prompt_template}
            )
        return research_qa_chain

    def get_urls(self, articles):
        urls = []
        try:
            urls.append(articles["answerBox"]["link"])
        except:
            pass
        for i in range(0, min(3, len(articles["organic"]))):
            urls.append(articles["organic"][i]["link"])
        return urls

    def get_content_from_urls(self, urls):
        loader = UnstructuredURLLoader(urls=urls)
        research_content = loader.load()
        return research_content

    def research_given_query(self, research_objective, research_content):

        docs = self.text_splitter.split_documents(research_content)
        self.db = FAISS.from_documents(documents=docs, embedding=self.hfembeddings)
        bot = self.research_answerer()
        research_out =bot({"query": research_objective})
        return research_out["result"]

    def research(self, query):
        search_articles = self.search_articles(query)
        urls = self.get_urls(search_articles)
        research_content = self.get_content_from_urls(urls)
        answer = self.research_given_query(query, research_content)
        return answer

In [13]:
!pip install streamlit_chat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00


In [21]:
# Initialize `clicked` in `st.session_state` if it doesn't already exist
if 'clicked' not in st.session_state:
    st.session_state.clicked = False

@st.cache_resource(show_spinner=True)
def create_researcher():
    researcher = Researcher()
    return researcher

research_apprentice = create_researcher()

def display_conversation(history):
    for i in range(len(history["apprentice"])):
        message(history["user"][i], is_user=True, key=str(i) + "_user")
        message(history["apprentice"][i], key=str(i))

st.title("RoboWiz - Your 24/7 AI Research Apprentice 🧑‍💻")
st.subheader("An AI apprentice who can serve you 24/7 by researching on a given question in realtime and provide you answers accordingly")

if "apprentice" not in st.session_state:
    st.session_state["apprentice"] = ["Hello. How can I help you?"]
if "user" not in st.session_state:
    st.session_state["user"] = ["Hey RoboWiz!"]

col1, col2 = st.columns([1,2])

with col1:
    st.image("https://images.unsplash.com/photo-1518770660439-4636190af475?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1350&q=80")
  # Ensure the image path is correct or use a URL

with col2:
    with st.expander("Command RoboWiz"):
        research_query_input = st.text_input("Research Query")
        if st.button("Send"):
            st.session_state.clicked = True  # Update `clicked` to True when the button is pressed
            robowiz_output = research_apprentice.research(research_query_input)

            st.session_state["user"].append(research_query_input)
            st.session_state["apprentice"].append(robowiz_output)

            display_conversation(st.session_state)
